In [1]:
#!pip install pandas==1.3.5

In [2]:
import pandas as pd
import sys
import os 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import string
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType
import warnings

from recommenders.utils.spark_utils import start_or_get_spark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


warnings.simplefilter(action='ignore', category=FutureWarning)

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Spark version: 3.1.2


In [3]:
pd.__version__

'1.3.5'

In [47]:
ratings_data = pd.read_csv(
        r"C:\Users\mirza\Desktop\dataset\MovieLens-1M\ratings.csv",sep = "::",
        names=["userId", "movieId", "ratings", "timestamp"])

movie= pd.read_csv(
        r"C:\Users\mirza\Desktop\dataset\MovieLens-1M\movies.csv",sep = "::",
        names=["movieId", "Title", "Genres"])

C:\Users\mirza\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [48]:
movie

,movieId,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [49]:
ratings_data

,userId,movieId,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [50]:
 spark = SparkSession \
  .builder \
  .appName("Example") \
  .config("spark.driver.memory", "16g")\
  .getOrCreate()

In [51]:
ratings_data=spark.createDataFrame(ratings_data) 
#movie=spark.createDataFrame(movie) 

In [52]:
for field in ratings_data.schema.fields:
    print(field.name +" , "+str(field.dataType))

userId , LongType
movieId , LongType
ratings , LongType
timestamp , LongType


In [53]:
(training, test) = ratings_data.randomSplit ([0.8, 0.2])

als = ALS(userCol="userId", itemCol="movieId", ratingCol="ratings", coldStartStrategy="drop", nonnegative=True)

In [54]:
param_grid = ParamGridBuilder()\
            .addGrid(als.rank, [10])\
            .addGrid(als.maxIter, [15])\
            .addGrid(als.regParam, [0.05])\
            .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                               predictionCol="prediction")

tvs = TrainValidationSplit(estimator = als,
                          estimatorParamMaps= param_grid,
                          evaluator = evaluator)


In [55]:
model = tvs.fit(training)

best_model = model.bestModel

predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

print("RMSE = " + str(rmse))
print("**Best Model**")
print(" Rank:"), best_model.rank
print(" MaxIter:"), best_model._java_obj.parent().getMaxIter()
print(" RegParam:"), best_model._java_obj.parent().getRegParam()

RMSE = 0.858963540253887
**Best Model**
 Rank:
 MaxIter:
 RegParam:


(None, 0.05)

In [56]:
user_recs = best_model.recommendForAllUsers(5)

In [57]:
user_recs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{572, 5.125716},...|
|    27|[{670, 5.2344}, {...|
|    28|[{557, 4.7238846}...|
|    31|[{572, 4.9166923}...|
|    34|[{572, 5.4904385}...|
|    44|[{3092, 4.957605}...|
|    53|[{557, 5.994532},...|
|    65|[{572, 6.3049498}...|
|    76|[{572, 5.5808873}...|
|    78|[{3415, 5.0223584...|
|    81|[{557, 5.5025225}...|
|    85|[{1796, 6.006885}...|
|   101|[{2388, 6.6718373...|
|   108|[{296, 4.3056245}...|
|   115|[{887, 6.480039},...|
|   126|[{2776, 6.3038836...|
|   133|[{3314, 6.223572}...|
|   137|[{572, 5.321805},...|
|   148|[{572, 5.3418393}...|
|   155|[{572, 5.020179},...|
+------+--------------------+
only showing top 20 rows



In [63]:
movie["Genres"]= movie["Genres"].str.replace('[{}]'.format(string.punctuation), ", ")
movie["movieId"]= movie["movieId"].str.replace('[{}]'.format(string.punctuation), '')
movie["movieId"]=movie["movieId"].astype(int)


In [67]:
def get_recs_for_user(user):
    recs = user_recs.filter(user_recs.userId==user)
    recs = recs.select("recommendations.movieId", "recommendations.rating") # add "userId" ıf you want to add userId column
    movies = recs.select("movieId").toPandas().iloc[0,0]
    #ratings= recs.select("rating").toPandas().iloc[0,0]
    #userıd= recs.select("userId").toPandas().iloc[0,0] 
    ratings_matrix= pd.DataFrame(movies, columns=["movieId"])
    #ratings_matrix["ratings"]=ratings
    #ratings_matrix["userId"]=userıd  
    #ratings_matrix_ps= spark.createDataFrame(ratings_matrix) #ıf you want to get spark dataframe
    matrix = pd.merge(ratings_matrix, movie, how="inner", on="movieId")
    

    recommended_movies = []
    recommended_movies_posters = []
    #recommended_genres = []
    
    for i in matrix.movieId:
        recommended_movies_posters.append(i)
        
    for k in matrix.Title:
        recommended_movies.append(k)
        
   # for m in matrix.Genres:
   #     recommended_genres.append(m)
        
    
    return recommended_movies, recommended_movies_posters#, recommended_genres

In [68]:
get_recs_for_user(125)

(['Foreign Student (1994)',
  'Dingo (1992)',
  "All the Rage (a.k.a. It's the Rage) (1999)",
  'Raiders of the Lost Ark (1981)',
  'Saving Private Ryan (1998)'],
 [572, 853, 3867, 1198, 2028])

In [69]:
import pickle

pickle.dump(movie, open("movie.pkl","wb" ))
pickle.dump(users, open("users.pkl","wb" ))

In [70]:
user_recs.select("userId", "recommendations").write.save("user_recs.parquet")